In [ ]:
import boto3
from sagemaker import get_execution_role
import time

sagemaker_client = boto3.client('sagemaker')
role = get_execution_role()

model_name = 'simple-linear-regression-model'
bucket_name = 'justin-lambda-functions-bucket'
model_artifact_path = f's3://{bucket_name}/model/model.tar.gz'
container_image = '683313688378.dkr.ecr.us-west-2.amazonaws.com/linear-learner:latest'  # Example container for linear regression

# Create a SageMaker model
response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image,
        'ModelDataUrl': model_artifact_path
    },
    ExecutionRoleArn=role
)
print("Model created:", response)

# Create an endpoint configuration
endpoint_config_name = 'simple-linear-regression-endpoint-config'
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InstanceType': 'ml.m4.xlarge',
            'InitialInstanceCount': 1
        }
    ]
)
print("Endpoint configuration created:", response)

# Create an endpoint
endpoint_name = 'simple-linear-regression-endpoint'
response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
print("Endpoint creation initiated:", response)

# Wait for the endpoint to be in service
status = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
while status['EndpointStatus'] != 'InService':
    print('Endpoint status:', status['EndpointStatus'])
    time.sleep(30)
    status = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)

print('Endpoint status:', status['EndpointStatus'])
